In [4]:
!pip install -r requirements.txt > /dev/null

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
from langame_client import LangameClient
from pprint import pprint

In [2]:
langame_client = LangameClient()

In [5]:
# https://emojipedia.org/search
langame_client.generate_save_memes_general(topics_with_emojis = [
    ("artificial intelligence", "🤔🧠💭📚📖📐🧬🤖🦾🦿"),
    ("transhumanism", "🤔🧠💭📚📖📐🧬🤖🦾🦿"),
],
    memes_per_topic = 3,
    wikipedia_description = True,
    openai_description = True,
    anti_rate_limit_delay = 1)

call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is artificial intelligence, this is a list of starters to ask to other persons:
1. 
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\u00a0What is your first impression of artificial intelligence?\n2. \u00a0Do you think that we can create an artificial intelligence more intelligent than human beings? Why or why not? \u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0 \u00a0\u00a0\n3. What do you think about the future of Artificial Intelligence in 2020: what are the trends for this area and how will influence our daily life?\n4. Which film, book or TV program on AI impressed you most and why did it impress you so much? How would a movie like \"The Terminator\" appear to be if it really happened today ?\n5. Do 

call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is transhumanism, this is a list of starters to ask to other persons:
1. 
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\u00a0Do you think that there is a possibility to improve human body? If yes, how do you want to be improved?\n2. \u00a0What are the positive and negative aspects of transhumanism according to you?\n3. \u00a0How can we manage the consequences of this technological improvement from a moral and spiritual point of view?\n4. \u00a0Who will have access by technology, and who won't? Would this kind or inequality between people increase or decrease after becoming more \"effective\"? Who would benefit in such an unequal society: those who already had access by technology before it became widely

[(DatetimeWithNanoseconds(2021, 6, 9, 20, 6, 0, 834348, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f7b41866070>),
 (DatetimeWithNanoseconds(2021, 6, 9, 20, 6, 12, 327306, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f7b773b8be0>),
 (DatetimeWithNanoseconds(2021, 6, 9, 20, 6, 23, 839087, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f7b41861dc0>),
 (DatetimeWithNanoseconds(2021, 6, 9, 20, 6, 40, 844332, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f7b4186f5e0>),
 (DatetimeWithNanoseconds(2021, 6, 9, 20, 7, 5, 753470, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f7b410d72e0>),
 (DatetimeWithNanoseconds(2021, 6, 9, 20, 7, 14, 624997, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7f7b418a07f0>)]

In [7]:
# https://emojipedia.org/search
pprint(langame_client.generate_save_memes_general(topics_with_emojis = [
    ("philosophy", "🤔🧠💭📚📖"),
],
    memes_per_topic = 1,
    memes_per_meme=3,
    wikipedia_description = True,
    openai_description = True,
    anti_rate_limit_delay = 1))

call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is philosophy, this is a list of starters to ask to other persons:
1.
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " What is the best thing you have learned about philosophy?\n2. Which philosophical problem interests you most? Why do you choose this one and not any other?\n3. What book that is based on a philosophical concept has changed your life the most? Tell me, why did it influence you so much.\n4. Do you believe in anything which cannot be proven by science or logic (i.e., faith)? How can we determine whether something is true if we cannot prove it with evidence and reasoning (logic)?"
    }
  ],
  "created": 1623512761,
  "id": "cmpl-3A07d4uM5pADrdANSYAbazZ1ft6SL",
  "model": "davinci:2020-05-03",

In [ ]:
# https://emojipedia.org/search
langame_client.generate_save_memes_general(topics_with_emojis = [
    ("philosophy", "🤔🧠💭📚📖"),
    ("mathematics", "🤔🧠💭📚📖📐"),
    ("sciences", "🤔🧠💭📚📖📐"),
    ("physics", "🤔🧠💭📚📖📐"),
    ("biology", "🤔🧠💭📚📖📐🧬"),
],
    memes_per_topic = 3,
    memes_per_meme=3,
    wikipedia_description = True,
    openai_description = True,
    anti_rate_limit_delay = 1)

call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is philosophy, this is a list of starters to ask to other persons:
1.
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " What is your philosophy of life?\n2. What it means to be a philosopher?\n3. Do you think that everything has its purpose in life?\n"
    }
  ],
  "created": 1623512934,
  "id": "cmpl-3A0AQfU7c4UPKP9lreBnHqJt2YGT1",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
calling openai_description with prompt Below is a long paragraph which describes the topic of 'philosophy' as following:
call_completion with prompt Below is a long paragraph which describes the topic of 'philosophy' as following:
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
   

calling openai_description with prompt Below is a long paragraph which describes the topic of 'mathematics' as following:
call_completion with prompt Below is a long paragraph which describes the topic of 'mathematics' as following:
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n\"Mathematics is the study of quantity and structure through abstraction, an organized collection of all known mathematical facts. It includes the theories and techniques involved in working with numbers, shapes or formulas."
    }
  ],
  "created": 1623513011,
  "id": "cmpl-3A0BfmoCIqjdkOMARoA5t0P050ZtN",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is mathematics, this is a list of starters to ask to othe

In [6]:
# https://emojipedia.org/search
pprint(langame_client.generate_save_memes_ice_break(memes=5, memes_per_meme=3))

call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that break the ice between people.
1.What did school help to kill in you?
2.Ulysses bucket list: "Tell me something you have done, or want to do, that you think I should do? It can be anything, as challenging as you want it to be, or as easy. As long as you give me the rest of my life to complete it, I promise I will do it."
3.
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " If you could have a superpower, what would it be?\n4. What was the worst or best thing that happened to you this year?\n"
    }
  ],
  "created": 1623512725,
  "id": "cmpl-3A073vTjnOkylMBt4cvECcVaC0sVH",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that break th

In [11]:
from google.protobuf.json_format import MessageToDict, ParseDict
from langame.protobuf.langame_pb2 import Meme, Tag
from firebase_admin import firestore

tags = langame_client._firestore_client.collection_group(u"tags")\
    .where(u"topic.content", u"in", ["ice break"])\
    .order_by("createdAt", direction=firestore.Query.DESCENDING)\
    .limit(1)\
    .stream()
for t in tags:
    pprint(t.reference.parent.parent.get().to_dict())

{'content': 'hat do you like about your city?\n'
            'What’s something that happened to you recently that surprised '
            'you?',
 'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 29, 9, 459000, tzinfo=<UTC>)}


In [26]:
def string_ordered_list_to_list(my_string):
    return [e for e in my_string.split("\n")]

In [60]:
memes = langame_client._firestore_client.collection(u"memes").stream()
import re
for m in memes:
    filtered = string_ordered_list_to_list(m.to_dict().get("content"))
    new = []
    for e in filtered:
        cleaned = re.sub("^[0-9]*.\s", "", e, 1)
        new.append(cleaned.strip() if cleaned else e)
    print(m.reference.update({u"content": "\n".join(new)}))


update_time {
  seconds: 1623510732
  nanos: 204758000
}

update_time {
  seconds: 1623510835
  nanos: 769901000
}

update_time {
  seconds: 1623510835
  nanos: 953013000
}

update_time {
  seconds: 1623073336
  nanos: 935521000
}

update_time {
  seconds: 1623510836
  nanos: 260301000
}

update_time {
  seconds: 1623510836
  nanos: 438192000
}

update_time {
  seconds: 1623510836
  nanos: 608893000
}

update_time {
  seconds: 1623073393
  nanos: 544705000
}

update_time {
  seconds: 1623510836
  nanos: 948429000
}

update_time {
  seconds: 1623073112
  nanos: 557666000
}

update_time {
  seconds: 1623073082
  nanos: 241545000
}

update_time {
  seconds: 1623072958
  nanos: 472576000
}

update_time {
  seconds: 1623510837
  nanos: 573785000
}

update_time {
  seconds: 1623510837
  nanos: 748683000
}

update_time {
  seconds: 1623510837
  nanos: 915495000
}

update_time {
  seconds: 1623510838
  nanos: 54019000
}

update_time {
  seconds: 1623510838
  nanos: 214855000
}

update_time {
 

In [43]:
print(re.sub("^[0-9]*.\s", " ", "3. Is there something that scares you in science? For ", 1))

 Is there something that scares you in science? For 
